# Siamese Neural Network (SNN) (Testing)

## Importing libraries

In [ ]:
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.image as mpimg
import torchvision.utils
import numpy as np
import random
import pandas as pd
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from math import sqrt
from operator import itemgetter

## Configuration class

In [ ]:
class Config:
    
    def __init__(self, training_dir, testing_dir, dir_training, dir_testing):
        self.training_dir = training_dir
        self.testing_dir = testing_dir
        self.dset_training = pd.read_csv(dir_training, index_col = 0)
        self.dset_testing = pd.read_csv(dir_testing, index_col = 0)

conf = Config("./data/memes/training/", 
              "./data/memes/testing/",
              "./data/spb_training.csv", 
              "./data/spb_testing.csv")

normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                 std = [0.229, 0.224, 0.225])

## Loading model

In [ ]:
class SiameseNetwork_VGG16(nn.Module):
    
    def __init__(self, use_pretrained, num_classes):
        super(SiameseNetwork_VGG16, self).__init__()
        self.model_ft = models.vgg16(pretrained = use_pretrained)
        num_ftrs = self.model_ft.classifier[0].in_features
        self.model_ft = nn.Sequential(*list(self.model_ft.children())[:-1])
        self.fc_mse = nn.Sequential(
            nn.Dropout(),
            nn.Linear(num_ftrs * 2, num_classes, bias = True),
            nn.ReLU(inplace = True),
            nn.Linear(num_classes, 1, bias = True)
        )

    def forward_once(self, x):
        output = self.model_ft(x)
        return output.view(output.size(0), -1)

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        output = torch.cat((output1, output2), 1)
        output = self.fc_mse(output)
        return output

In [ ]:
net = SiameseNetwork_VGG16(True, 512).cuda()
net.load_state_dict(torch.load("./models/vgg16_mse.zip"))
net.eval()

## Dataset class

In [ ]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self, imageFolderDataset, dset_csv, transform = None):
        self.imageFolderDataset = imageFolderDataset
        self.dset_csv = dset_csv
        self.transform = transform
        
    def __getitem__(self, index):  
        img_tuple = self.imageFolderDataset.imgs[index] 
        img_id = self.obtain_id(img_tuple[0])
        img_fav = self.dset_csv.loc[img_id]['favorites']
        img_tensor = Image.open(img_tuple[0])
        img_tensor = img_tensor.convert("RGB")
        if self.transform is not None:
            img_tensor = self.transform(img_tensor)
        return img_id, img_tensor, img_fav

    def __len__(self):
        return len(self.imageFolderDataset.imgs)

    def obtain_id(self, img_route):
        return int(img_route.split("/")[-1].split(".")[0])

## Loading training dataset

In [ ]:
tr_folder_dataset = dset.ImageFolder(conf.training_dir)
tr_siamese_dataset = SiameseNetworkDataset(imageFolderDataset = tr_folder_dataset,
                                        dset_csv = conf.dset_training,
                                        transform = transforms.Compose([transforms.Resize((224, 224)),
                                                    transforms.ToTensor(), 
                                                    normalize]))
tr_dataloader = DataLoader(tr_siamese_dataset, num_workers = 2, batch_size = 1, shuffle = True)

## Loading testing dataset

In [ ]:
ver_folder_dataset = dset.ImageFolder(conf.testing_dir)
ver_siamese_dataset = SiameseNetworkDataset(imageFolderDataset = ver_folder_dataset,
                                        dset_csv = conf.dset_testing,
                                        transform = transforms.Compose([transforms.Resize((224, 224)),
                                                    transforms.ToTensor(), 
                                                    normalize]))
ver_dataloader = DataLoader(ver_siamese_dataset, num_workers = 2, batch_size = 1, shuffle = True)

## Creating tensors

In [ ]:
def create_tensors(dataloader, dataset):
    ids, tensors, favs = [], [], []
    data = iter(dataloader)
    for _ in range(len(dataset)):
        id, img, fav = next(data)
        ids.append(id.item())
        tensors.append(img)
        favs.append(fav.item())
    df = pd.DataFrame(index = ids, data = {'tensors': tensors, 'favorites': favs})
    return df   

In [ ]:
tr_tensors = create_tensors(tr_dataloader, tr_siamese_dataset)
ver_tensors = create_tensors(ver_dataloader, ver_siamese_dataset)

In [ ]:
def plot_histogram(data, hist_title):
    fig = plt.figure(figsize = (20, 10))
    bins = np.arange(min(data), max(data) + 250, 250)
    n, bins, patches = plt.hist(data, bins = bins, histtype = 'step', align = 'left', facecolor = 'blue')
    plt.xlabel('Likes', fontsize = 20)
    plt.xlim([min(data), max(data)])
    plt.ylabel('Memes', fontsize = 20)
    plt.title(hist_title, fontsize = 25)
    plt.show()

## Likes prediction

In [ ]:
def likes_prediction(tr_tensors, ver_tensors, net, n):
    ids, original, calculated = [], [], []
    for i in ver_tensors.index:
        likes = []
        for j in tr_tensors.index:
            fav = tr_tensors.loc[j]['favorites']
            dif = net(ver_tensors.loc[i]['tensors'].cuda(), tr_tensors.loc[j]['tensors'].cuda())
            likes.append([j, fav, abs(dif.item())])
        likes.sort(key = itemgetter(2))
        likes, y = likes[:n], 0
        for x in likes:
            y += x[1] 
        ids.append(likes)
        original.append(ver_tensors.loc[i]['favorites'])
        calculated.append(int(y/n))
    df = pd.DataFrame(index = ver_tensors.index, data = {'ids': ids, 'original': original, 'calculated': calculated})  
    return df

In [ ]:
vgg16_pred = likes_prediction(tr_tensors, ver_tensors, net, 5)

In [ ]:
def ecm_rmse(tensors, likes_range):
    ecm, n, p = 0, 0, 0
    for i in tensors.index:
        if tensors.loc[i]['original'] in range(likes_range[0], likes_range[1] + 1):
            ecm += (tensors.loc[i]['original'] - tensors.loc[i]['calculated'])**2
            if tensors.loc[i]['calculated'] >= 0.9*tensors.loc[i]['original'] and tensors.loc[i]['calculated'] <= 1.1*tensors.loc[i]['original']:
                p += 1
            n += 1
    ecm = round(ecm/n, 2)
    rmse = round(sqrt(ecm), 2)
    positives = round(p/n, 2)
    return 'Range({}, {})\n ECM = {}, RMSE = {}, Acc = {}'.format(likes_range[0], likes_range[1], ecm, rmse, positives)

In [ ]:
print(ecm_rmse(vgg16_pred, [750, 1000]))
print(ecm_rmse(vgg16_pred, [1001, 1500]))
print(ecm_rmse(vgg16_pred, [1501, 2000]))
print(ecm_rmse(vgg16_pred, [2001, int(max(vgg16_pred['original']))]))

## Visual plot (memes and best match)

In [ ]:
def plot_memes(likes_df, n):
    images, j = [], 0
    for i in likes_df.index:
        route = "./memes/testing/testing/{}.jpeg".format(i)
        images.append([route, likes_df.loc[i]['original']])
        route = "./memes/training/training/{}.jpeg".format(likes_df.loc[i]['ids'][0][0])
        images.append([route, likes_df.loc[i]['ids'][0][1], likes_df.loc[i]['ids'][0][2]])
        j += 1
        if j == n:
            break
    fig = plt.figure(figsize = (10, n * 5))
    for i in range(n * 2):
        plt.subplot(n, 2, i + 1)
        read_img = mpimg.imread(images[i][0])
        plt.imshow(read_img)
        plt.axis('off')
        if (i + 1) % 2 == 0:
            plt.title('{} likes (Dif = {} likes)'.format(images[i][1], round(images[i][2], 2)))
        else:
            plt.title('{} likes'.format(images[i][1]))
    plt.show()

In [ ]:
plot_memes(vgg16_pred, 5)